In [53]:
import torch
from torch.nn import functional as F
import math

d = 1024
T = 128
p=0.0
dropout = torch.nn.Dropout(p=p)

device=torch.device('cuda')
x = torch.randn(T, d).to(device)

# dropout=.5
wq = torch.rand(d,d).to(device)
wk = torch.rand(d,d).to(device)
wv = torch.rand(d,d).to(device)

q = x@wq
k = x@wk
v = x@wv

mask = ~torch.tril(torch.ones(T, T).to(torch.bool)).to(device)  # causal mask

In [54]:
# %timeit 
z = F.scaled_dot_product_attention(q,k,v, dropout_p=p,is_causal=True)#.sum(1)

In [55]:
# %%timeit

attn = q @ k.transpose(-2, -1)  # B,nh,T,C @ B,N,C,T = B,nh,T,T
attn = torch.masked_fill(attn, mask, value=float('-inf'))
attn = attn * math.sqrt(k.size(-1))  # causal mask, normalize
attn = torch.softmax(attn, dim=-1)
dattn = dropout(attn)
# self.last_attn_map = attn.detach().cpu().clone()
# assert dattn.isnan().sum() == 0
z2 = dattn @ v  # B,nh,T,T @ B,nh,T,hs = B,nh,T,hs


In [56]:
z

tensor([[ -0.2110,  -1.5818,  -5.6437,  ..., -18.7893,  10.4426,   7.2090],
        [ -0.2110,  -1.5818,  -5.6437,  ..., -18.7893,  10.4426,   7.2090],
        [-24.8567, -26.1446, -23.0648,  ..., -21.2818, -12.6791, -14.6870],
        ...,
        [ 35.5712,  44.4382,  53.2575,  ...,  54.2117,  36.7921,  40.5944],
        [ 35.5712,  44.4382,  53.2575,  ...,  54.2117,  36.7921,  40.5944],
        [-46.2574, -32.8182, -45.1698,  ..., -28.5072, -20.4379, -13.8146]],
       device='cuda:0')

In [57]:
z2

tensor([[ -0.2110,  -1.5818,  -5.6437,  ..., -18.7893,  10.4426,   7.2090],
        [ -0.2110,  -1.5818,  -5.6437,  ..., -18.7893,  10.4426,   7.2090],
        [-24.8567, -26.1446, -23.0648,  ..., -21.2818, -12.6791, -14.6870],
        ...,
        [ 35.5712,  44.4382,  53.2575,  ...,  54.2117,  36.7921,  40.5944],
        [ 35.5712,  44.4382,  53.2575,  ...,  54.2117,  36.7921,  40.5944],
        [-46.2574, -32.8182, -45.1698,  ..., -28.5072, -20.4379, -13.8146]],
       device='cuda:0')